(var_ode_sys)=

# Variational ODEs

```{versionadded} 5.0.0

```

Consider a system of differential equations in the standard form

$$
\frac{d\boldsymbol{x}}{dt} = f\left(\boldsymbol{x}, \boldsymbol{\alpha}, t\right),
$$

where $\boldsymbol{x}$ is the vector of state variables and $\boldsymbol{\alpha}$ a vector of parameters. For a given set of initial conditions $\boldsymbol{x}_0$ at time $t_0$, the solution of this system will be

$$
\boldsymbol{x} = \boldsymbol{x}\left(t, \boldsymbol{x}_0, t_0, \boldsymbol{\alpha} \right).
$$

When solving numerically initial-value problems, it is often useful to compute not only the solution, but also its partial derivatives with respect to the initial conditions and/or the parameters. The derivatives with respect to the initial conditions, for instance, are needed for the computation of [chaos indicators](https://en.wikipedia.org/wiki/Lyapunov_exponent) and for [uncertainty propagation](https://en.wikipedia.org/wiki/Propagation_of_uncertainty), and they can also be used to propagate a small neighborhood in phase space around the initial conditions. The derivatives with respect to the parameters of the system are required when formulating optimisation and inversion problems such as orbit determination, trajectory optimisation and training of neural networks in [neural ODEs](./NeuralODEs.ipynb).

There are two main methods for the computation of the partial derivatives. The first one is based on the application of automatic differentiation (AD) techniques directly to the numerical integration algorithm. This can be done either by replacing the algebra of floating-point numbers with the algebra of (generalised) [dual numbers](https://en.wikipedia.org/wiki/Dual_number) (aka truncated Taylor polynomials), or via [differentiable programming](https://en.wikipedia.org/wiki/Differentiable_programming) techniques. The former approach is used by libraries such as [pyaudi](https://github.com/darioizzo/audi), [desolver](https://github.com/Microno95/desolver) and [TaylorIntegration.jl](https://docs.sciml.ai/TaylorIntegration/stable/jet_transport/), while differentiable programming is popular in the [Julia programming language](https://en.wikipedia.org/wiki/Julia_(programming_language)) community.

The second method is based on the formulation of the *variational equations*, that is, differential equations satisfied by the partial derivatives which are added to and solved together with the original ODEs. For instance, we can formulate differential equations for the first-order derivatives with respect to the initial conditions via elementary calculus:

$$
\frac{d}{dt}\frac{\partial \boldsymbol{x}}{\partial \boldsymbol{x}_0} = \frac{\partial }{\partial \boldsymbol{x}_0} \frac{d \boldsymbol{x}}{dt} = \frac{\partial f}{\partial \boldsymbol{x}_0} = \frac{\partial f}{\partial \boldsymbol{x}} \frac{\partial \boldsymbol{x}}{\partial \boldsymbol{x}_0}.
$$

The variational ODE system then reads

$$
\begin{cases}
\frac{d\boldsymbol{x}}{dt} = f\left(\boldsymbol{x}, \boldsymbol{\alpha}, t\right) \\
\frac{d}{dt}\frac{\partial \boldsymbol{x}}{\partial \boldsymbol{x}_0} = \frac{\partial f}{\partial \boldsymbol{x}} \frac{\partial \boldsymbol{x}}{\partial \boldsymbol{x}_0}
\end{cases},
$$

and the original state vector $\boldsymbol{x}$ has been extended to include the variational state variables $\frac{\partial \boldsymbol{x}}{\partial \boldsymbol{x}_0}$.

heyoka.py adopts the variational approach for the computation of the partial derivatives, supporting the formulation of variational ODEs at arbitrary differentiation orders and with respect to any combination of initial conditions, parameters and initial time. In this tutorial, we will explore this feature and show a couple of interesting use cases.

Before beginning, however, let us point out for clarity (and for the benefit of the search engines indexing this page) that in the scientific literature there is a bewildering variety of different names and monikers used when discussing partial derivatives of ODEs and their applications. Here is a (partial) list:

- in the astrodynamics community, the term *differential algebra* is often used to refer to the computation of partial derivatives via truncated Taylor polynomials (e.g., see [this paper](https://link.springer.com/article/10.1007/s10569-010-9283-5)). The term actually originates from the community of beam physics, where it has been used in the context of the theoretical modelling of particle accelerators since the 90s (e.g., see [this review](https://www.bmtdynamics.org/pub/papers/DAHAPE12/DAHAPE12.pdf));
- in the mathematical community, the term *jet transport* is sometimes used to refer to the propagation of a small neighborhood in phase space around the initial conditions via the Taylor series constructed form the partial derivatives (e.g., see [this paper](http://www.maia.ub.es/~angel/varis/granada09.pdf)). In heyoka.py, we refer to a similar idea as {ref}`Taylor map evaluation <taylor_map>`;
- in the Julia programming language community, the term *local sensitivity analysis* refers to the computation of the partial derivatives via the variational equations, while *discrete sensitivity analysis* refers to the computation of the partial derivatives by directly differentiating the numerical method's steps (e.g., see [this review](https://arxiv.org/abs/1812.01892));
- in the space engineering community, the term *state transition tensors* is sometimes used to indicate the generalisations of the [state transition matrix](https://en.wikipedia.org/wiki/State-transition_matrix) (which in turn is built from the first-order partial derivatives) to higher differentiation orders.

## Constructing a variational ODE system

Let us begin with the definition of a simple ODE system:

In [1]:
import heyoka as hy

# Create the symbolic variables x and v.
x, v = hy.make_vars("x", "v")

# Create an ODE system.
sys = [(x, v), (v, hy.cos(hy.time) - hy.par[0] * v - hy.sin(x))]

This is the forced damped pendulum system already considered in [another tutorial](<./Non-autonomous systems.ipynb>), where we have introduced the air friction coefficient as the [runtime parameter](<./ODEs with parameters.ipynb>) ``par[0]``.

We then proceed to create a {class}`~heyoka.var_ode_sys`

In [2]:
vsys = hy.var_ode_sys(sys, hy.var_args.vars, order=2)

In [3]:
vsys.order

2

In [12]:
vsys.sys

[(x, v),
 (v, ((cos(t) - (p0 * v)) - sin(x))),
 (∂[(0, 1)]x, ∂[(0, 1)]v),
 (∂[(1, 1)]x, ∂[(1, 1)]v),
 (∂[(0, 1)]v, (-(∂[(0, 1)]x * cos(x)) - (∂[(0, 1)]v * p0))),
 (∂[(1, 1)]v, (-(∂[(1, 1)]x * cos(x)) - (∂[(1, 1)]v * p0))),
 (∂[(0, 2)]x, ∂[(0, 2)]v),
 (∂[(0, 1), (1, 1)]x, ∂[(0, 1), (1, 1)]v),
 (∂[(1, 2)]x, ∂[(1, 2)]v),
 (∂[(0, 2)]v,
  (-(∂[(0, 2)]v * p0) - (((∂[(0, 1)]x * -sin(x)) * ∂[(0, 1)]x) + (∂[(0, 2)]x * cos(x))))),
 (∂[(0, 1), (1, 1)]v,
  (-(∂[(0, 1), (1, 1)]v * p0) - (((∂[(0, 1)]x * -sin(x)) * ∂[(1, 1)]x) + (∂[(0, 1), (1, 1)]x * cos(x))))),
 (∂[(1, 2)]v,
  (-(∂[(1, 2)]v * p0) - (((∂[(1, 1)]x * -sin(x)) * ∂[(1, 1)]x) + (∂[(1, 2)]x * cos(x)))))]

(taylor_map)=
## Taylor map evaluation